Strictly speaking, they are not convolutions but cross-correlations.

In [2]:
import tensorflow as tf
import numpy as np

sess = tf.Session()
sess.run(tf.global_variables_initializer())

BATCH_SIZE = 1
HEIGHT = 4
WIDTH = 4
CHANNELS = 1

In [3]:
M = np.array([
    [[[1], [2], [3], [4]],
     [[0], [1], [2], [3]],
     [[0], [0], [1], [2]],
     [[0], [0], [0], [1]]]],
    dtype=np.float32)

Define some filters we can use for conv2d with the above matrix.

In [3]:
filter_diagonal = np.array(
    [
        [[[1]], [[0]]],
        [[[0]], [[1]]]
    ],
    dtype=np.float32)

filter_vertical = np.array(
    [
        [[[1]], [[-1]]],
        [[[1]], [[-1]]]
    ],
    dtype=np.float32)

Now try `conv2d` on `M` with these filters:

In [11]:
inputs_tf = tf.placeholder(tf.float32, shape=[BATCH_SIZE, HEIGHT, WIDTH, CHANNELS], name='input')
outputs_tf = [tf.nn.conv2d(inputs_tf, filter=f, strides=[1, 2, 1, 1], padding='VALID')
              for f in [filter_diagonal, filter_vertical]]

a, b = sess.run(outputs_tf, {inputs_tf: M})

In [6]:
a

array([[[[ 2.],
         [ 4.],
         [ 6.]],

        [[ 0.],
         [ 0.],
         [ 2.]]]], dtype=float32)

In [7]:
b

array([[[[-2.],
         [-2.],
         [-2.]],

        [[ 0.],
         [-1.],
         [-2.]]]], dtype=float32)

We see that the first filter (`filter_diagonal`) sums the diagonal of the 2x2 submatrices and the other one rewards values where the left side is positive and the right side is non-positive.

The `SAME` padding extends the input when we reach the edge.:

In [15]:
sess.run(tf.nn.conv2d(inputs_tf, filter=filter_diagonal, strides=[1, 2, 1, 1], padding='SAME'),
         {inputs_tf: M})

array([[[[ 2.],
         [ 4.],
         [ 6.],
         [ 4.]],

        [[ 0.],
         [ 0.],
         [ 2.],
         [ 2.]]]], dtype=float32)

The `VALID` padding computes the valid submatrices only, stopping when we reach the inside edge:

In [16]:
sess.run(tf.nn.conv2d(inputs_tf, filter=filter_diagonal, strides=[1, 2, 1, 1], padding='VALID'),
         {inputs_tf: M})

array([[[[ 2.],
         [ 4.],
         [ 6.]],

        [[ 0.],
         [ 0.],
         [ 2.]]]], dtype=float32)